In [1]:
import os
import pandas as pd

import logging
import time
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# NEEDS TO BE ABSOLUTE PATH
PATH_TO_DOWNLOADS = "/Users/mdong/dataScience/projects-ml/waste-management/calrecycle-data"
options = Options()
prefs = {'download.default_directory' : PATH_TO_DOWNLOADS}
options.add_experimental_option('prefs', prefs)
options.headless = False

> path to downloads folder needs to EMPTY before running, or at least should be no files with the default download name

how to iterate through each county? 

- each county doesn't map to URL like on waste characterization page
- use the n-th child selector - one for each county, starts at 2 + 58 counties
- document.querySelector("#CountyID > option:nth-child(59)")

In [2]:
# for county_idx in range(2, 60):
#     print(county_idx)

In [5]:
def get_waste_origin_data(idx, download_path):
    """Exports excel file with origin waste data for the county specified by idx to download_path
    """
    DATA_URL = "https://www2.calrecycle.ca.gov/LGCentral/DisposalReporting/Origin/CountywideSummary"
    driver = webdriver.Chrome(options=options)
    driver.get(DATA_URL)
    
    county_selector = '#CountyID > option:nth-child({})'.format(idx)
    county_element = driver.find_element_by_css_selector(county_selector)
    county = county_element.text
    county_element.click()

    driver.find_element_by_css_selector('#SearchButton').click()

    default_name_download = PATH_TO_DOWNLOADS + "/CountywideSummary.xlsx"
    print("Downloading data for county: {}".format(county))

    seconds_waited = 0
    while not os.path.exists(default_name_download):
        time.sleep(1)
        seconds_waited += 1

    print("Download took {} seconds...".format(seconds_waited))
    updated_file_name = download_path + "/" + county + "_waste_production.xlsx"
    
    if os.path.isfile(default_name_download):
        os.rename(default_name_download, updated_file_name)
        try:
            os.remove(default_name_download)
        except:
            pass

    driver.quit()

In [6]:
get_waste_origin_data(6, PATH_TO_DOWNLOADS)

Download took 1 seconds...


### get it working for one county

In [11]:
DATA_URL = "https://www2.calrecycle.ca.gov/LGCentral/DisposalReporting/Origin/CountywideSummary"

driver = webdriver.Chrome(options=options)

driver.get(DATA_URL)

county_element = driver.find_element_by_css_selector('#CountyID > option:nth-child(5)')
county = county_element.text
county_element.click()

driver.find_element_by_css_selector('#SearchButton').click() # .send_keys("\n")

most_recent_waste_data = PATH_TO_DOWNLOADS + "/CountywideSummary.xlsx"
print("Downloading data for county: {}".format(county_element.text))

seconds_waited = 0

while not os.path.exists(most_recent_waste_data):
    time.sleep(1)
    seconds_waited += 1

print("Download took {} seconds...".format(seconds_waited))

driver.quit()

Download took 1 seconds...


In [ ]:
def convert_to_csv(path):
    """Convert the Excel file into a csv with the same name given path to the file
    """
    directory = os.path.dirname(path)
    file_name_w_ext = os.path.basename(path)
    file_name = os.path.splitext(file_name_w_ext)[0]
    csv_file_path = os.path.join(directory, file_name + ".csv")
    
    data_xls = pd.read_excel(path)
    data_xls.to_csv(csv_file_path, index=False)

In [ ]:
convert_to_csv()